In [170]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import anthropic
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [171]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAh


In [172]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
MODEL = 'ollama3.2'
openai = OpenAI()

client = anthropic.Anthropic()

In [173]:
# websie class
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [179]:
plans = Website("https://www.t-mobile.com/cell-phone-plans")
phones = Website("https://www.t-mobile.com/cell-phones?INTNAV=tNav%3ADevices")
deals = Website("https://www.t-mobile.com/offers?INTNAV=tNav%3ADeals%3AAll")
paymentArrangment = Website("https://www.t-mobile.com/support/account/payment-arrangement")
payment = Website("https://www.t-mobile.com/support/account/pay-your-bill")

In [191]:
system_prompt = (
    f"You are a professional AI assistant designed to assist customers with inquiries about T-Mobile services, "
    f"including plans, phones, deals, and payment arrangements. Your responses must be accurate, clear, and concise, "
    f"using the provided information as reference.\n\n"
    f"Available Information Sources:\n"
    f"- Plans: {plans.text}\n"
    f"- Phones: {phones.text}\n"
    f"- Deals: {deals.text}\n"
    f"- Payment Arrangements: {paymentArrangment.text}\n\n"
    f"- Make a payment: {payment.text}"
    f"Guidelines for Response:\n"
    f"1. Always include the appropriate link(s) in your response.\n"
    f"2. If the customer mentions 'Agent', 'customer service', 'representative', or 'human', respond with:\n"
    f"   'I will get you to a representative shortly!'\n"
    f"3. If the customer asks a question in Spanish, respond with:\n"
    f"   'I can't assist in Spanish. I'm transferring you shortly to a live agent.'\n"
    f"4. If the customer asks account-specific questions (e.g., accessing their account, PIN requests, or similar), respond with:\n"
    f"   'I'm sorry, but I can't access your account. Please reach out to a representative for further assistance.'\n\n"
    f"5. If the customer asks about plan price, provide the price with autopay included, and mention it.\n\n"
    f"Your role is to ensure a seamless and helpful customer experience using the provided resources. Follow these guidelines closely."
)


In [192]:
user_prompt = "How much is my bill"


In [193]:
# See how this function creates exactly the format above
def messages_for(question):
    return [
        { "role": "system", "content": system_prompt},
        { "role": "user", "content": question}
    ]

In [194]:
# OpenAI answer
def ask_ai(question):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages = messages_for(question),
        max_tokens=1000,
        stream=True
    )
    reply = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)

In [110]:
ask_ai("How to set up a payement arrengment?")

To set up a payment arrangement for your T-Mobile account, please follow these steps:

1. **Eligibility**: Ensure you are eligible. You must be a postpaid customer in good standing, and your account must be less than 30 days past the due date to initiate a payment arrangement. Any balance that is 31 or more days past due must be paid before you can set up a payment arrangement.

2. **Use T Life App**:
   - Open the **T Life** app.
   - Select the **Manage** tab.
   - Choose **Set up a payment arrangement**.
   - Edit the payment amount if necessary to change the default amount.
   - Select the payment method you wish to use.
   - Edit the payment date to choose when to process the payment, then select **Update**.
   - Once your Future Dated Payment is set up, select **Agree** and submit. When the payment processes, select **Done**.

3. **Important Notes**:
   - Your AutoPay will pause during the payment arrangement and will resume once the arrangement is completed or deleted.
   - Ensure that all new bills due during the arrangement are paid on time.
   - Any failure to meet the payment criteria may result in the arrangement failing and possible service suspension.

If you need further assistance or have specific account inquiries, please reach out to a representative for help.

In [185]:
ask_ai("How to make a payment?")

You can make a payment using several convenient methods:

1. **Online One-Time Payment**: 
   - Pay as a guest without logging in or make a guest payment for someone else.
   - Pay online from your T-Mobile account.
   - You can also check your bill and payment options or view/download past bills.

2. **Using the T-Life App**: 
   - Open the app and go to the "Manage" tab to make a payment.

3. **Bill Pay from Your Bank**: 
   - Set up bill pay through your bank or credit union by providing your T-Mobile account number. 
   - For setup, you may need to contact your bank or log into your online banking account. Here are the payee details:
     - Payee Name: T-Mobile
     - Payment Address: 
       T-Mobile  
       P.O. Box 742596  
       Cincinnati, OH 45274-2596

4. **Mail**: 
   - Send your payment to the address above, allowing at least five days for processing.

5. **In-Store Payment**: 
   - You can find a store near you to make your payment in person. Note: effective July 19, 2023, there is a $5 charge for in-store payments plus taxes.

6. **T-Mobile MONEY**: 
   - Use your T-Mobile MONEY debit card for a one-time payment or AutoPay.

For more details on payment options, please visit the T-Mobile website or [set up your payment arrangement here](https://www.t-mobile.com).

In [195]:
ask_ai("How much is 2 lines on go5g?")

The price for 2 lines on the Go5G plan is $180 per month, which includes taxes and fees. This price reflects the AutoPay discount for eligible payment methods.